In [ ]:
import pandas as pd
import numpy as np    

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/python csv files/Human stress detection.csv')
df.head()

,Humidity,Temperature,Step count,Stress Level
0,21.33,90.33,123.0,1.0
1,21.41,90.41,93.0,1.0
2,27.12,96.12,196.0,2.0
3,27.64,96.64,177.0,2.0
4,10.87,79.87,87.0,0.0


In [ ]:
df.columns

Index(['Humidity', 'Temperature', 'Step count', 'Stress Level'], dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48730 entries, 0 to 48729
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Humidity      10299 non-null  float64
 1   Temperature   10299 non-null  float64
 2   Step count    10299 non-null  float64
 3   Stress Level  10299 non-null  float64
dtypes: float64(4)
memory usage: 1.5 MB


In [ ]:
df.describe()

,Humidity,Temperature,Step count,Stress Level
count,10299.000000,10299.000000,10299.000000,10299.000000
mean,19.996456,86.995745,100.330226,1.105447
std,5.784539,11.450249,58.079644,0.771248
min,10.000000,36.580000,0.000000,0.000000
25%,15.000000,83.390000,50.000000,0.000000
50%,19.980000,88.590000,102.000000,1.000000
75%,24.985000,93.800000,150.000000,2.000000
max,56.890000,99.000000,200.000000,2.000000


In [ ]:
df.isnull().sum()

Humidity        38431
Temperature     38431
Step count      38431
Stress Level    38431
dtype: int64

In [ ]:
df['Humidity'] = df['Humidity'].fillna(df['Humidity'].mean())
df['Temperature'] = df['Temperature'].fillna(df['Temperature'].mean())
df['Step count'] = df['Step count'].fillna(df['Step count'].mean())
df['Stress Level'] = df['Stress Level'].fillna(df['Stress Level'].mean())

In [ ]:
df.isnull().sum()

Humidity        0
Temperature     0
Step count      0
Stress Level    0
dtype: int64

In [ ]:
df.corr()

,Humidity,Temperature,Step count,Stress Level
Humidity,1.000000,0.081326,0.677491,0.729512
Temperature,0.081326,1.000000,0.075795,0.084417
Step count,0.677491,0.075795,1.000000,0.648414
Stress Level,0.729512,0.084417,0.648414,1.000000


In [ ]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of         Humidity  Temperature  Step count  Stress Level
0      21.330000    90.330000  123.000000      1.000000
1      21.410000    90.410000   93.000000      1.000000
2      27.120000    96.120000  196.000000      2.000000
3      27.640000    96.640000  177.000000      2.000000
4      10.870000    79.870000   87.000000      0.000000
...          ...          ...         ...           ...
48725  19.996456    86.995745  100.330226      1.105447
48726  19.996456    86.995745  100.330226      1.105447
48727  19.996456    86.995745  100.330226      1.105447
48728  19.996456    86.995745  100.330226      1.105447
48729  19.996456    86.995745  100.330226      1.105447

[48730 rows x 4 columns]>

In [ ]:
df.to_csv('pre_data.csv')

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('frido').getOrCreate()

In [ ]:
df1 = spark.read.csv('pre_data.csv', inferSchema=True, header=True)


In [ ]:
df1.show()

+--------+-----------+----------+------------+
|Humidity|Temperature|Step count|Stress Level|
+--------+-----------+----------+------------+
|   21.33|      90.33|       123|           1|
|   21.41|      90.41|        93|           1|
|   27.12|      96.12|       196|           2|
|   27.64|      96.64|       177|           2|
|   10.87|      79.87|        87|           0|
|   11.31|      80.31|        40|           0|
|   18.16|      87.16|        88|           1|
|    28.2|       97.2|       162|           2|
|   14.25|      83.25|        61|           0|
|   26.13|      95.13|       168|           2|
|   23.61|      92.61|       200|           2|
|   19.37|      88.37|       117|           1|
|   29.08|      98.08|       179|           2|
|   17.83|      86.83|        55|           1|
|   28.06|      97.06|       148|           2|
|   19.43|      88.43|       123|           1|
|   26.85|      95.85|       169|           2|
|   26.51|      95.51|       135|           2|
|   29.49|   

In [ ]:
df1.printSchema()

root
 |-- Humidity: double (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Step count: integer (nullable = true)
 |-- Stress Level: integer (nullable = true)



In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
featureassem = VectorAssembler(inputCols=["Humidity","Temperature","Step count"],outputCol="Features")

In [ ]:
output=featureassem.transform(df1)

In [ ]:
output.show()

+--------+-----------+----------+------------+-------------------+
|Humidity|Temperature|Step count|Stress Level|           Features|
+--------+-----------+----------+------------+-------------------+
|   21.33|      90.33|       123|           1|[21.33,90.33,123.0]|
|   21.41|      90.41|        93|           1| [21.41,90.41,93.0]|
|   27.12|      96.12|       196|           2|[27.12,96.12,196.0]|
|   27.64|      96.64|       177|           2|[27.64,96.64,177.0]|
|   10.87|      79.87|        87|           0| [10.87,79.87,87.0]|
|   11.31|      80.31|        40|           0| [11.31,80.31,40.0]|
|   18.16|      87.16|        88|           1| [18.16,87.16,88.0]|
|    28.2|       97.2|       162|           2|  [28.2,97.2,162.0]|
|   14.25|      83.25|        61|           0| [14.25,83.25,61.0]|
|   26.13|      95.13|       168|           2|[26.13,95.13,168.0]|
|   23.61|      92.61|       200|           2|[23.61,92.61,200.0]|
|   19.37|      88.37|       117|           1|[19.37,88.37,117

In [ ]:
final_otp = output.select("Stress Level","Features")

In [ ]:
train_data,test_data = final_otp.randomSplit([0.75,0.25])

In [ ]:
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator        
regressor = RandomForestClassifier(featuresCol='Features',labelCol="Stress Level")
regressor=regressor.fit(train_data)

In [ ]:
reg_predictions = regressor.transform(test_data)

In [ ]:
predictions = regressor.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol="Stress Level",
    predictionCol="prediction",
    metricName="accuracy")
acc = evaluator.evaluate(predictions)
y_true = predictions.select(['Stress Level']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print('The random forest had an accuracy of: {0:2.2f}%'.format(acc*100))

The random forest had an accuracy of: 99.25%


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator        
regressor = DecisionTreeClassifier(featuresCol='Features',labelCol="Stress Level")
regressor=regressor.fit(train_data)

In [ ]:
predictions = regressor.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol="Stress Level",
    predictionCol="prediction",
    metricName="accuracy")
acc = evaluator.evaluate(predictions)
y_true = predictions.select(['Stress Level']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print('The Decision tree had an accuracy of: {0:2.2f}%'.format(acc*100))

The Decision tree had an accuracy of: 99.63%


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator        
regressor = LogisticRegression(featuresCol='Features',labelCol="Stress Level")
regressor=regressor.fit(train_data)

In [ ]:
predictions = regressor.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol="Stress Level",
    predictionCol="prediction",
    metricName="accuracy")
acc = evaluator.evaluate(predictions)
y_true = predictions.select(['Stress Level']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print('The logistic regression had an accuracy of: {0:2.2f}%'.format(acc*100))

The logistic regression had an accuracy of: 100.00%


In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import MulticlassClassificationEvaluator        
regressor = GBTRegressor(featuresCol='Features',labelCol="Stress Level")
regressor=regressor.fit(train_data)

In [ ]:
predictions = regressor.transform(test_data)
evaluator = MulticlassClassificationEvaluator(
    labelCol="Stress Level",
    predictionCol="prediction",
    metricName="accuracy")
acc = evaluator.evaluate(predictions)
y_true = predictions.select(['Stress Level']).collect()
y_pred = predictions.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print('The Gradient boost had an accuracy of: {0:2.2f}%'.format(acc*100))

The Gradient boost had an accuracy of: 91.01%
